## Associação com Apriori
- Trabalho realizado para a minha **pós-graduação em Inteligência Artificial**

#### **Objetivo**: encontrar associações entre os itens mais frequentes da base de dados *Iris*.

### Parte 1 - Aplicação do Apriori nos dados originais

In [1]:
#pip install apyori

In [2]:
# Bibliotecas
import pandas as pd       
import numpy as np
import matplotlib.pyplot as plt  
from apyori import apriori

**Convertendo o dataset Iris para o formato de lista, necessário para aplicar o Apriori**

In [3]:
transactions = []

with open('data/iris.csv') as dataset:
    for line in dataset:
        transaction = [item for item in line.strip().split(',') if item != 'NaN']
        transactions.append(transaction)
    
transactions[:3]

[['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'],
 ['5.1', '3.5', '1.4', '0.2', 'setosa'],
 ['4.9', '3', '1.4', '0.2', 'setosa']]

**Aplicando a função Apriori**

In [4]:
# Requisitos mínimos de Suporte estabelecidos em 7% e de Confiança em 50%

rules = list(apriori(
    transactions, 
    min_support=0.07, 
    min_confidence=0.50,
    min_length=2))

rules_df = pd.DataFrame(
    [{'De': list(rule[0])[0],
    'Para': list(rule[0])[1],
    'Support': rule[1],
    'Confidence': rule[2][0][2],
    'Lift': rule[2][0][3]} for rule in rules if len(rule[0]) == 2])
rules_df = rules_df.dropna()

rules_df = rules_df.sort_values('Support', ascending=False).head(20)
rules_df.head(10)

,De,Para,Support,Confidence,Lift
0,0.2,setosa,0.185430,1.000000,3.020000
3,setosa,1.5,0.092715,0.538462,1.626154
1,1.3,versicolor,0.086093,0.650000,1.963000
2,setosa,1.4,0.079470,0.600000,1.812000
4,1.8,virginica,0.072848,0.916667,2.768333


#### Comentário:
- Para aplicar o Apriori sobre os dados originais e encontrar algum resultado, parametrizei as medidas de suporte e confiança em 0.07 e 0.50, respectivamente. Neste cenário, vemos que a classe **setosa** aparece associada aos itens 0.2, 1.5 e 1.4; a classe **versicolor** ao item 1.3; e a classe **virginica** ao item 1.8. No entanto, esse resultado não é muito informativo, pois não me permitiu tirar nenhuma conclusão relevante.

### Parte 2 - Aplicação do Apriori nos dados discretizados

In [5]:
# Base de dados Iris original
df = pd.read_csv('data/iris.csv')
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


**Discretizando os dados de acordo com os % de seus quantis mínimo, 50%, 75% e máximo**

In [6]:
bin1 = [4.3, 5.8, 6.4, 7.9]
bin2 = [2.0, 3.0, 3.3, 4.4]
bin3 = [1.0, 4.35, 5.1, 6.9]
bin4 = [0.1, 1.3, 1.8, 2.5]

sepal_lth = pd.cut(df['sepal_length'].to_numpy(), bins=bin1, labels=["sep_lth_bxo", "sep_lth_md", "sep_lth_alto"])
sepal_wh = pd.cut(df['sepal_width'].to_numpy(), bins=bin2, labels=["sep_wth_bxo", "sep_wth_md", "sep_wtd_alto"])
petal_lth = pd.cut(df['petal_length'].to_numpy(), bins=bin3, labels=["pet_lth_bxo", "pet_lth_md", "pet_lth_alto"])
petal_wh = pd.cut(df['petal_width'].to_numpy(), bins=bin4, labels=["pet_wth_bxo", "pet_wth_md", "pet_wth_alto"])

In [7]:
df["sepal_lth"] = sepal_lth
df["sepal_wh"] = sepal_wh
df["petal_lth"] = petal_lth
df["petal_wh"] = petal_wh

In [8]:
df.head(1)

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_lth,sepal_wh,petal_lth,petal_wh
0,5.1,3.5,1.4,0.2,setosa,sep_lth_bxo,sep_wtd_alto,pet_lth_bxo,pet_wth_bxo


In [9]:
df = df[["species", "sepal_lth", "sepal_wh", "petal_lth", "petal_wh"]]

In [10]:
df["classe_sepal_length"] = df["species"].astype(str) + " " + df["sepal_lth"].astype(str)
df["classe_sepal_width"] = df["species"].astype(str) + " " + df["sepal_wh"].astype(str)
df["classe_petal_length"] = df["species"].astype(str) + " " + df["petal_lth"].astype(str)
df["classe_petal_width"] = df["species"].astype(str) + " " + df["petal_wh"].astype(str)
df.head(1)

,species,sepal_lth,sepal_wh,petal_lth,petal_wh,classe_sepal_length,classe_sepal_width,classe_petal_length,classe_petal_width
0,setosa,sep_lth_bxo,sep_wtd_alto,pet_lth_bxo,pet_wth_bxo,setosa sep_lth_bxo,setosa sep_wtd_alto,setosa pet_lth_bxo,setosa pet_wth_bxo


**Novo dataframe Iris**

In [11]:
df = df[["classe_sepal_length", "classe_sepal_width", "classe_petal_length", "classe_petal_width"]]
df.head(1)

,classe_sepal_length,classe_sepal_width,classe_petal_length,classe_petal_width
0,setosa sep_lth_bxo,setosa sep_wtd_alto,setosa pet_lth_bxo,setosa pet_wth_bxo


In [12]:
df.tail(1)

,classe_sepal_length,classe_sepal_width,classe_petal_length,classe_petal_width
149,virginica sep_lth_md,virginica sep_wth_bxo,virginica pet_lth_md,virginica pet_wth_md


In [13]:
df.to_csv('iris_association.csv', header=False, index=False)

**Conversão dos dados para lista e função Apriori**

In [14]:
transactions2 = []

with open('iris_association.csv') as df2:
    for line in df2:
        transaction2 = [item for item in line.strip().split(',') if item != 'NaN']
        transactions2.append(transaction2)
    
transactions2[:3]

[['setosa sep_lth_bxo',
  'setosa sep_wtd_alto',
  'setosa pet_lth_bxo',
  'setosa pet_wth_bxo'],
 ['setosa sep_lth_bxo',
  'setosa sep_wth_bxo',
  'setosa pet_lth_bxo',
  'setosa pet_wth_bxo'],
 ['setosa sep_lth_bxo',
  'setosa sep_wth_md',
  'setosa pet_lth_bxo',
  'setosa pet_wth_bxo']]

In [15]:
# Com os dados tratados, o requisito mínimo de Suporte foi estabelecido em 17% e o de Confiança em 70%

rules2 = list(apriori(
    transactions2, 
    min_support=0.17, 
    min_confidence=0.70,
    min_length=2))

rules_df2 = pd.DataFrame(
    [{'De': list(rule[0])[0],
      'Para': list(rule[0])[1],
      'Support': rule[1],
      'Confidence': rule[2][0][2],
      'Lift': rule[2][0][3]} for rule in rules2 if len(rule[0]) == 2])
rules_df2 = rules_df2.dropna()

### Resultado dos itens com associações mais frequentes:

In [16]:
rules_df2 = rules_df2.sort_values('Support', ascending=False)
rules_df2.head(10)

,De,Para,Support,Confidence,Lift
1,setosa sep_lth_bxo,setosa pet_lth_bxo,0.320000,0.979592,2.998751
3,setosa sep_lth_bxo,setosa pet_wth_bxo,0.293333,1.000000,3.061224
0,setosa pet_wth_bxo,setosa pet_lth_bxo,0.286667,0.877551,2.991651
5,setosa sep_lth_bxo,setosa sep_wtd_alto,0.200000,1.000000,3.061224
2,setosa sep_wtd_alto,setosa pet_lth_bxo,0.193333,0.966667,2.959184
4,setosa pet_wth_bxo,setosa sep_wtd_alto,0.193333,0.966667,3.295455
6,versicolor pet_wth_bxo,versicolor sep_wth_bxo,0.180000,0.964286,3.527875
7,virginica pet_wth_alto,virginica pet_lth_alto,0.173333,0.764706,3.373702


#### Comentários:
- Testando os dados da base com medidas de suporte >= 0.20 e confiança >= 0.70, somente a espécie **setosa** apresentou associações relevantes, sendo a mais frequente a que combina **setosa petal length baixo** com **setosa sepal length baixo**, com 0.32 de suporte e 0.98 de confiança. 
- As espécies **versicolor** e **virginica** só começaram a aparecer na lista de itens frequentes quando a medida de suporte foi reduzida para 0.17, mantendo a de confiança em 0.70.